In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from fastai.distributed import *
from fastai.vision.all import *
from pathlib import Path

module_path = Path('../../baselines').resolve(); assert module_path.exists()

if module_path not in sys.path: sys.path.append(str(module_path))

In [3]:
module_path

Path('/home/agrawalp2/prepositions/experiments/baselines')

In [4]:
device = torch.device('cuda:0')
core_pth = Path('/home/agrawalp2/prepositions'); assert core_pth.exists()

vidvrd_path = core_pth/Path('real_world_data/vidvrd/vidvrd-dataset'); assert vidvrd_path.exists()
encoder_path = core_pth/Path('experiments/baselines/models/encoder/GoogleNews-vectors-negative300.bin.gz'); assert encoder_path.exists()
stupd_path = Path('/data/dataset/agrawalp2/stupd/stupd_dataset'); assert stupd_path.exists()


spatialsenses_pth = core_pth/Path('real_world_data/spatialsense'); assert spatialsenses_pth.exists()


In [5]:
from pretraining_dataloaders.stupd.spatialsense.languageOnlyDataset import languageOnlyDataset
from pretraining_dataloaders.stupd.spatialsense.utils import split_dataset
from models.static.language_only import SimpleLanguageOnlyModel

In [6]:
ds = languageOnlyDataset(stupd_path/'annotations', encoder_path)
train_ds, valid_ds = split_dataset(ds, pct = 0.8)

train_dl = DataLoader(train_ds, batch_size =64 , shuffle = True, drop_last = True)
valid_dl = DataLoader(valid_ds, batch_size = 128 , shuffle = True, drop_last = True)

In [7]:
dls = DataLoaders(train_dl, valid_dl)
dls.n_inp = 2
model = SimpleLanguageOnlyModel(word_embedding_dim=300, feature_dim=512, c=train_ds.c).cuda()
learn = Learner(dls, model = model, loss_func = CrossEntropyLossFlat(), metrics = [accuracy,BalancedAccuracy()],
               path = core_pth/'experiments/baselines/weights',
               model_dir = 'languageOnly',
               )

In [8]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,accuracy,balanced_accuracy_score,time
0,1.326436,1.342949,0.382523,0.387592,00:08
1,1.283688,1.274017,0.399884,0.404015,00:08


In [9]:
learn.save('scratch_finetuneStupd')

Path('/home/agrawalp2/prepositions/experiments/baselines/weights/languageOnly/scratch_finetuneStupd.pth')

In [10]:
from dataloaders.spatialsense.languageOnlyDataset import languageOnlyDataset
from dataloaders.spatialsense.utils import  map_spatialsenses_to_stupd

In [11]:

train_ds = languageOnlyDataset(annotations_path = spatialsenses_pth/'annotations.json',
                               split = 'train',
                               encoder_path = encoder_path,
                               x_tfms = None, 
                               y_tfms = [map_spatialsenses_to_stupd],)

valid_ds = languageOnlyDataset(annotations_path = spatialsenses_pth/'annotations.json',
                               split = 'valid',
                               encoder_path = encoder_path,
                               x_tfms = None,
                               y_tfms = [map_spatialsenses_to_stupd])

In [12]:
train_dl = DataLoader(train_ds, batch_size =64 , shuffle = True, drop_last = True)
valid_dl = DataLoader(valid_ds, batch_size = 128 , shuffle = True, drop_last = True)

In [13]:
dls = DataLoaders(train_dl, valid_dl)
dls.n_inp = 2
model = SimpleLanguageOnlyModel(word_embedding_dim=300, feature_dim=512, c=train_ds.c).cuda()
learn = Learner(dls, model = model, loss_func = CrossEntropyLossFlat(), metrics = [accuracy,BalancedAccuracy()],
               path = core_pth/'experiments/baselines/weights',
               model_dir = 'languageOnly',)

In [16]:
learn.load('scratch_finetuneStupd', device = device)

In [17]:
learn.fit(5)

epoch,train_loss,valid_loss,accuracy,balanced_accuracy_score,time
0,1.691085,1.571814,0.401563,0.285310,00:01
1,1.507867,1.551732,0.418750,0.329991,00:01
2,1.391761,1.555239,0.449219,0.357878,00:01
3,1.290512,1.609280,0.438281,0.357787,00:01
4,1.184857,1.667590,0.427344,0.363479,00:01
